# Máquinas de Boltzman restringidas

In [4]:
# Importar las librerias
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [5]:
# importar el dataset
movies = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/movies.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
movies.head(3)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [6]:
movies.shape

(3883, 3)

In [7]:
users = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/users.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
users.head(3)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [8]:
ratings = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/ratings.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
ratings.head(3)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [9]:
# Preparar el conjunto de entrenamiento y de testing
training_set = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.base', sep = '\t')
training_set = np.array(training_set, dtype = 'int')

In [10]:
test_set = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.test', sep = '\t')
test_set = np.array(test_set, dtype = 'int')

In [11]:
# Obtener el numero de usuarios y de peliculas
number_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
number_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))

print('El numero de usuarios es {}'.format(number_users))
print('El numero de peliculas es {}'.format(number_movies))

El numero de usuarios es 943
El numero de peliculas es 1682


In [12]:
# Convertir los datos en un array X[u, i] con usuarios u en fila y peliculas i en columna

def convert(data):
  new_data = []
  for id_user in range(1, number_users + 1):
    id_movies = data[:,  1 ][data[:, 0] == id_user ]
    id_ratings = data[:,  2 ][data[:, 0] == id_user ]
    ratings = np.zeros(number_movies)
    ratings[id_movies - 1] = id_ratings 
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [13]:
# Convertir los datos a tensores de Torch

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [14]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [15]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### Crear la arquitectura de la Red Neuronal

In [22]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(number_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, number_movies)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [23]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

### Entrenar el SAE

In [24]:
nb_epoch = 200
for epoch in range(1, nb_epoch):
  train_loss = 0
  s = 0.
  for id_user in range(number_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae.forward(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = number_movies/float(torch.sum(target.data > 0 ) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data * mean_corrector)
      s += 1
      optimizer.step()
  print('Epoch: ' + str(epoch) + ', loss: ' + str(train_loss / s))


Epoch: 1, loss: tensor(1.7709)
Epoch: 2, loss: tensor(1.0966)
Epoch: 3, loss: tensor(1.0534)
Epoch: 4, loss: tensor(1.0384)
Epoch: 5, loss: tensor(1.0307)
Epoch: 6, loss: tensor(1.0266)
Epoch: 7, loss: tensor(1.0239)
Epoch: 8, loss: tensor(1.0218)
Epoch: 9, loss: tensor(1.0208)
Epoch: 10, loss: tensor(1.0193)
Epoch: 11, loss: tensor(1.0189)
Epoch: 12, loss: tensor(1.0183)
Epoch: 13, loss: tensor(1.0180)
Epoch: 14, loss: tensor(1.0175)
Epoch: 15, loss: tensor(1.0173)
Epoch: 16, loss: tensor(1.0169)
Epoch: 17, loss: tensor(1.0168)
Epoch: 18, loss: tensor(1.0163)
Epoch: 19, loss: tensor(1.0164)
Epoch: 20, loss: tensor(1.0160)
Epoch: 21, loss: tensor(1.0161)
Epoch: 22, loss: tensor(1.0158)
Epoch: 23, loss: tensor(1.0158)
Epoch: 24, loss: tensor(1.0157)
Epoch: 25, loss: tensor(1.0155)
Epoch: 26, loss: tensor(1.0154)
Epoch: 27, loss: tensor(1.0153)
Epoch: 28, loss: tensor(1.0150)
Epoch: 29, loss: tensor(1.0125)
Epoch: 30, loss: tensor(1.0118)
Epoch: 31, loss: tensor(1.0095)
Epoch: 32, loss: 


### Evaluar en el conjunto de test

In [28]:
nb_epoch = 200

test_loss = 0
s = 0.
for id_user in range(number_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae.forward(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = number_movies/float(torch.sum(target.data > 0 ) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s += 1
print('Test loss: ' + str(test_loss / s))

Test loss: tensor(0.9512)
